<a href="https://colab.research.google.com/github/pwesquire/mb_colab/blob/main/Profanity_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import user's Google Drive
# You will need /content/drive/MyDrive/formatted_bad_names.csv (https://drive.google.com/file/d/1pobWSRbL5ihp2fgd1hxH1sbyiFBgUZ3L/view?usp=sharing)
# and /content/drive/MyDrive/international_names.csv (https://drive.google.com/file/d/1B-gziiuVsGewAPhSBWZO5gk1rCiwHisN/view?usp=sharing)
# and /content/drive/MyDrive/labeled_names.csv (https://drive.google.com/file/d/1csnUvTnD4AQ-XHZT7SLVuagEnj08O9Wp/view?usp=sharing)
# which Patrick can share with you.
#
# Click the drive icon in the top right and "Add Shortcut to Drive" in the root of your
# Google Drive directory and this should all run
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import tensorflow as tf
# !pip install better_profanity
# from better_profanity import profanity as pf 
from pathlib import Path
!pip install -q -U "tensorflow-text==2.8.*"
import tensorflow_text as tft
import tensorflow_hub as hub
!pip install -q tf-models-official==2.7.0
from official.nlp import optimization 
# import functools
import matplotlib.pyplot as plt

# Set log level
tf.get_logger().setLevel('ERROR')

# Panda options for outputting tables
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# Creating Panda DataFrame objects from our CSV files
bad_names = pd.read_csv("/content/drive/MyDrive/formatted_bad_names.csv")
good_names = pd.read_csv("/content/drive/MyDrive/international_names.csv")

In [ ]:
# Fix some missing data in our csv files
# • Take random country codes from good_names and insert them into bad names list (due to missing country codes)
# • Transform bad names to title case
rand_good = good_names.sample(2224, ignore_index=True)
bad_names.fillna(rand_good, inplace=True)
bad_names_titlecase = bad_names.transform({"first_name": lambda x: x.title(), "last_name": lambda x: x.title(), "gender": lambda x: x, "country_code": lambda x: x})
print(bad_names_titlecase)

In [ ]:
# Apply label to indicate the profanity score of each row of the DataFrames (data sets)
good_names = good_names.assign(label=0.0)
bad_names = bad_names_titlecase.assign(label=1.0)

In [ ]:
# Combine good and bad names together
all_names = good_names.append(bad_names, ignore_index=True)

# Make sure there are no missing values
all_names.fillna({"first_name": "", "last_name": "", "country_code": "NA"}) # Note - NA is Namibia, change this

# Create full name field from first and last
all_names['full_name'] = all_names[['first_name','last_name']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Method to test profanity filter library - Deprecated
# Could we do label_profanity_mb(first_name, last_name) and just mark anybody with "poop" as a name 1.0
# TODO: Bergen will find bad words list
# TODO: Patrick will build this in 5 min.
# Then we can do (pseudocode, obvs):
#   def label_profanity_for_dataFrame(incomingDataFrame) { for each df: label_profanity(df.first, df.last) }
#   def label_profanity(first_name, last_name) { ... }
#   ... then do ...
#   label_profanity_in_some_basic_way()
#   fancy=false
#   if fancy:
#     label_profanity_in_some_fancy_way()
#   return_or_set_profanity_score()
def label_profanity(name):
  if pf.contains_profanity(name):
    return 1.0
  else:
    return 0.0



# all_names['label'] = all_names[['full_name']].apply(lambda row: label_profanity(row['full_name']), axis=1)
# ^^ was profanity library attempt but took too long -- was about 3 seconds per row^^
# could maybe work for single submissions... 
# but would have to be a fleet of lambda's to handle 1000 submissions/hr

# removing unused columns/fields from dataFrame
all_names = all_names.drop(["first_name","last_name","gender"], axis=1)
# randomizing the rows
all_names_rand = all_names.reindex(np.random.permutation(all_names.index))

# Save a file with names labelled with a profanity score
# Possibly: in the future, could get rid of many previous steps and start with this labelled dataset
#   Could break out earlier code into a method like prepare_and_label_dataset()
filepath = Path('/content/drive/MyDrive/labeled_names.csv') 
all_names_rand.to_csv(filepath, index=False)  

In [ ]:
# Taking first X rows as dataset for training the model
train_df = all_names_rand.head(40000)
# Taking other rowns for validation and testing, respectively
val_df = all_names_rand.iloc[45001:50000]
test_df = all_names_rand.tail(5000)

In [ ]:
# Ensure that columns have the correct datatype
# TODO: streamline by doing all_names.astype()
train_df = train_df.astype({'full_name': 'string', 'country_code': 'string', 'label':'float32'})
val_df = val_df.astype({'full_name': 'string', 'country_code': 'string', 'label':'float32'})
test_df = test_df.astype({'full_name': 'string', 'country_code': 'string', 'label':'float32'})
# print(train_df['full_name'].dtype)
# print(all_names.tail(5))

In [ ]:
#
# From here on out it is a bit murky
#

# Remove column from dataFrame for use... so model doesn't see any label we've applied
train_labels = train_df.pop('label')
val_labels = val_df.pop('label')
test_labels = test_df.pop('label')

# tf.data.Dataset is one possible TensorFlow format that you can feed into a model
# We are passing labels in... doing something with 'full_name & label'
# Explanation from Patrick 9/26/22:
#   A Dataset is a bunch of Tensors
#   Each Tensor has a Shape and a DataType
#   Tensors in a Dataset can be arranged in many different structures
#   Gets confusing...
#   Every type of Tensor has a full gigantic article in api_docs... (https://www.tensorflow.org/api_docs/python/tf)
#     Can do with csv, raw data, so many ways
train_ds = tf.data.Dataset.from_tensor_slices((tf.strings.as_string(train_df['full_name']), train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((tf.strings.as_string(val_df['full_name']), val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((tf.strings.as_string(test_df['full_name']), test_labels))

# Autotune is something to improve the performance through caching
# These are just values from a tutorial
# AUTOTUNE _is_ a configurable buffer size, in the end
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Grabbing some predefined models (like a gem/library) from the TensorFlow Hub
# Usage:
#    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
#    preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
#    encoder_inputs = preprocessor(text_input)
# Explanation from PTorrez
#   Layer is a function that is one step in a model
#   Input layer is where your data enters a model
#   Data often has some processing before it goes into the model
#   Model is a sequence of layers
#   Nodes are weighted that affect how subsequent layers receive their inputs
#   A challenge is: How to process your data into a usable form to go into a model?
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

In [ ]:
# Creating a model with a bunch of layers in it
# Weighted nodes that define how (the data gets processed)
# Output of each layer gets fed into the next layer
# Sequence of layers:
#   input: receives string inputs
#   preprocessing: get vocabulary that is in the strings
#   encoder: encode as tokens and maps them to integers (or floats?)
#   net: puts those into a net with nodes
#     Dropout - deletes some of the input, maybe to improve performance (unclear - from tutorial)
#     Dense - convert into a float between zero and one* (*probably not true)
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(None,), dtype=tf.string, name='sentences')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)

  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)

  return tf.keras.Model(text_input, net)

In [ ]:
# Set up model
classifier_model = build_classifier_model()

# Configure how many rounds of weighting adjustments it goes through
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

# Learning Rate (a very tiny number)
# How much the weighting can (affect the data/change) each epoch(?)
init_lr = 3e-5

# Create optimizer based on this configuration
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# Loss - not sure 
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Metrics - not sure
metrics = tf.metrics.BinaryAccuracy()

# Compile model to create a sequence to feed data through
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

# Fit the model - calculate the weights to make the model be able to predict labels for data
# history is a variable... we don't use it but maybe for analysis for how well it's performing. Not sure.
# After calling model.fit you can call model.predict or model.evaluate. Not sure the difference.
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

In [ ]:
# Evaluate is an analysis tool to see how well model performs on a test dataset
# Gauge how well the model performs
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# Predict
# A good test of the model would be something like:
#    keras_model.predict(aSingleRowsWorthOfData) to see the label that it would predict



### Everything below here is from previous attempts ###

Next steps:

* Set up a very basic Profanity Filter API that doesn't even try to use an ML model
  * Patrick builds a Ruby "Profanity Filter" API on SENDY that, given a request {first,last,email, region,country,postal_code} will always return { score : .5 }
  * Bergen gets Patrick a dataset of bad words
  * Patrick updates the Profanity Filter API to, given a single request: 
    * If EITHER first or last name match a bad word, we give it a .9 and return that value to the caller. If not, we return a .1
* Set up a Python-based Profanity Detection Model API
  * First build it to return an error message and no value  { error : { message : "No profanity score returned" }  }
  * Then, we build it out to return a value and work on improving the model

In [ ]:

# train_labels = tf.constant(train_df['label'])

# Preprocess the input strings.
train_data = preprocess_data(train_ds)
val_data = preprocess_data(val_ds)
test_data = preprocess_data(test_ds)

# Build the Keras model.
keras_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[None], dtype=tf.int64),

    # Feature layer goes here?
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation(tf.nn.relu),
    tf.keras.layers.Dense(1)
])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
keras_model.compile(loss='binary_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])
keras_model.fit(train_data, train_labels, epochs=5,callbacks=[tensorboard_callback])
# print(keras_model.predict(test_data))

ValueError: ignored

In [ ]:
test_bad = pd.DataFrame(data={'full_name': "BJ CUMMINGS", 'country_code': 'CA'}, index=[0]).astype({'full_name': 'string', 'country_code': 'string'})
test_good = pd.DataFrame(data={'full_name': "Bjork Gonzo", 'country_code': 'JP'}, index=[0]).astype({'full_name': 'string', 'country_code': 'string'})
processed_bad = preprocess_data(test_bad)
processed_good = preprocess_data(test_good)

test_prediction_bad = keras_model.predict(processed_bad)
test_prediction_good = keras_model.predict(processed_good)

print("bad name prediction:",test_prediction_bad)

print("good name prediction:",test_prediction_good)

bad name prediction: [[-0.01348143]]
good name prediction: [[-0.01752897]]
